# Metadata extraction using DSPy and a local LLM using GEPA optimization

To run this, you first need to start two local vLLM servers in the backround. These commands are tested on a single A100 80GB in non-exclusive mode (e.g. Turso oversub GPU). The GPT-OSS 120B model has to be partially offloaded to CPU to preserve VRAM.

For the main extractor model:

    vllm serve google/gemma-3-4b-it --port 7987 --max-model-len 16384 --gpu-memory-utilization 0.25

For the reflection model:
    
    llama-server -hf ggml-org/gpt-oss-120b-GGUF --host 0.0.0.0 --port 7988 --ctx-size 0 --jinja -ub 2048 -b 2048 --n-cpu-moe 5


In [1]:
import dspy

MODEL_ID = "google/gemma-3-4b-it"  # should match the model vLLM is running (does it matter??)
PORT = 7987  # should match the port where vLLM is running
MAX_TOKENS = 1024  # limit on how many new tokens to generate (default: 4000)
TEMPERATURE = 0.7

lm = dspy.LM("openai/" + MODEL_ID,
             api_base=f"http://localhost:{PORT}/v1",  # ensure this points to your port
             api_key="local", model_type="chat", max_tokens=MAX_TOKENS, temperature=TEMPERATURE)
dspy.configure(lm=lm)

# test the connection to the LLM
lm("Who are you?", temperature=0.0)

["I'm Gemma, a large language model created by the Gemma team at Google DeepMind. I’m an open-weights model, which means I’m widely available for public use! \n\nI can take text and images as inputs and generate text-based responses. \n\nYou can learn more about me on the Gemma project page: [https://ai.google.dev/gemma](https://ai.google.dev/gemma)"]

In [2]:

REFLECTION_MODEL_ID = "ggml-org/gpt-oss-120b-GGUF"
REFLECTION_PORT = PORT + 1
REFLECTION_MAX_TOKENS = 7988

reflection_lm = dspy.LM("openai/" + REFLECTION_MODEL_ID,
             api_base=f"http://localhost:{REFLECTION_PORT}/v1",  # ensure this points to your port
             api_key="local", model_type="chat", max_tokens=REFLECTION_MAX_TOKENS, temperature=TEMPERATURE)

# test the connection to the LLM
reflection_lm("Who are you?", temperature=0.0)

['I’m ChatGPT\u202f—\u202fa large language model created by OpenAI. I’ve been trained on a wide variety of text up through June\u202f2024, which lets me help with things like answering questions, brainstorming ideas, explaining concepts, drafting or editing writing, solving problems, and much more. I don’t have personal experiences or consciousness, and I can’t browse the web in real time, but I can draw on the information I was trained on to generate useful, context‑aware responses.  \n\nIf there’s anything specific you’d like to know or discuss, just let me know!']

In [3]:
# Load and prepare dataset

import json
import glob
import random

random.seed(42)  # for deterministic sampling of validation set

train_files = glob.glob("../../llm-dataset/*-train.jsonl")
test_files = glob.glob("../../llm-dataset/*-test.jsonl")

VAL_SIZE = 64  # how many documents to validate on during optimization

def preprocess_sample(sample):
    # fix some bad field names
    ground_truth = { fld.replace('-', '_'): val for fld, val in sample["ground_truth"].items() }
    output = json.dumps(ground_truth)
    input_ = json.dumps(sample["content"])
    return dspy.Example({"content": input_, "metadata": output}).with_inputs("content")

def dataset_to_records(files):
    records = []
    for filename in files:
        with open(filename) as infile:
            for line in infile:
                sample = json.loads(line)
                records.append(preprocess_sample(sample))
    return records


train_val_set = dataset_to_records(train_files)
random.shuffle(train_val_set)

train_set = train_val_set[VAL_SIZE:]
val_set = train_val_set[:VAL_SIZE]

test_set = dataset_to_records(test_files)

len(train_set), len(val_set), len(test_set)

(576, 64, 182)

In [4]:
print("Input Message:")
print(train_set[-1]['content'])

print("\n\nGold Answer:")
for k, v in json.loads(train_set[-1]['metadata']).items():
    print(f"{k}: {v}")

Input Message:
{"pdfinfo": {"creationDate": "D:20201214215341+01'00'", "modDate": "D:20201214215418+01'00'"}, "pages": [{"page": 1, "text": "# ANTAA TAITEEN OPETTAA\n\n\n"}, {"page": 3, "text": "ANTA A TAITEEN OPETTA A GERT BIESTA\n\n\n"}, {"page": 4, "text": "00:00:08.18\n\n\n"}, {"page": 5, "text": "00:00:36.03 00:00:52.19 00:00:54.19\n\n\n"}, {"page": 6, "text": "00:00:58.16 00:01:00.17 00:01:0\n\n\n"}, {"page": 65, "text": "\u2018Opastan sinua kaikessa, n\u00e4yt\u00e4n sinulle kaiken ja nime\u00e4n kaiken.\u2019\n\u2014 COMENIUS\nT\u00e4ss\u00e4 kirjassa Gert Biesta esitt\u00e4\u00e4 uuden n\u00e4kemyksen nykyaikaisesta taidekasvatuksesta\n\nosoittamalla, ett\u00e4 taide tarjoaa ainutlaatuisia v\u00e4lineit\u00e4 olla dialogissa maailman kanssa. N\u00e4kemys\n\nperustuu ajatukseen, ett\u00e4 opettaminen on n\u00e4ytt\u00e4mist\u00e4. Opettaja n\u00e4ytt\u00e4\u00e4 oppilaalle millaisiin\n\nhyviin, t\u00e4rkeisiin tai merkitt\u00e4viin asioihin maailmassa voisi kiinnitt\u00e4\u00e4

In [5]:
from typing import Optional

class ExtractInfo(dspy.Signature):
    """Extract structured metadata from text extracted from a PDF."""

    content: str = dspy.InputField()
    language: str = dspy.OutputField(desc="The language of the resource expressed as a BCP47 language tag.")
    title: str = dspy.OutputField(desc="The main title of the publication.")
    alt_title: list[str] = dspy.OutputField(desc="Alternative or parallel titles of the publication, suffixed with a BCP47 language tag in curly brackets.")
    creator: list[str] = dspy.OutputField(desc="The primary author(s) of the resource (order: Last Name, First Names).")
    year: Optional[str] = dspy.OutputField(desc="The year on which the resource was issued or made available.")
    publisher: list[str] = dspy.OutputField(desc="The entity/entities responsible for making the resource available.")
    doi: Optional[str] = dspy.OutputField(desc="The Digital Object Identifier (DOI) associated with the resource.")
    e_isbn: list[str] = dspy.OutputField(desc="The ISBN associated with the electronic resource.")
    p_isbn: list[str] = dspy.OutputField(desc="The ISBN of the printed version of this document.")
    e_issn: Optional[str] = dspy.OutputField(desc="The ISSN associated with the electronic resource.")
    p_issn: Optional[str] = dspy.OutputField(desc="The ISSN of the printed version of this document.")
    type_coar: str = dspy.OutputField(desc="The type of the resource according to the COAR Resource Types classification.")

module = dspy.ChainOfThought(ExtractInfo)

text = "Apple Inc. announced its latest iPhone 14 today." \
    "The CEO, Tim Cook, highlighted its new features in a press release."
response = module(content=text)

print(response)


Prediction(
    reasoning='The text describes an announcement by Apple Inc. regarding the iPhone 14. The CEO, Tim Cook, is mentioned, indicating a press release. The information provided is sufficient to identify the main entities and the type of resource.',
    language='en',
    title='Apple Inc. Announces iPhone 14',
    alt_title=[],
    creator=['Apple Inc.', 'Tim Cook'],
    year=None,
    publisher=['Apple Inc.'],
    doi=None,
    e_isbn=[],
    p_isbn=[],
    e_issn=None,
    p_issn=None,
    type_coar='News Article'
)


In [6]:
import Levenshtein

ALMOST_THRESHOLD = 0.95  # Adjust as needed

def feedback_simple_string(field, true_val, pred_val):
    score = 1.0 if true_val == pred_val else 0.0
    if score == 1.0:
        feedback = f"✅ `{field}` is correct: `{true_val}`."
    else:
        feedback = f"❌ `{field}` is incorrect. You predicted `{pred_val}`, but the correct value is `{true_val}`."
    return score, feedback

def feedback_fuzzy_string(field, true_val, pred_val):
    base_score = 1.0 if true_val == pred_val else 0.0
    if base_score == 1.0 or (true_val and pred_val and Levenshtein.ratio(true_val.lower(), pred_val.lower()) >= ALMOST_THRESHOLD):
        score = 1.0
        feedback = f"✅ `{field}` is approximately correct: `{pred_val}` matches `{true_val}` closely."
    else:
        score = 0.0
        feedback = f"❌ `{field}` is incorrect. You predicted `{pred_val}`, but the correct value is `{true_val}`."
    return score, feedback

def feedback_set(field, true_val, pred_val):
    true_set = set(true_val or [])
    pred_set = set(pred_val or [])

    if not true_set and not pred_set:
        return 1.0, f"✅ `{field}` is empty as expected."
    elif not true_set or not pred_set:
        return 0.0, f"❌ `{field}` is incorrect. Expected `{true_set}`, but got `{pred_set}`."

    tp = len(true_set & pred_set)
    fp = len(pred_set - true_set)
    fn = len(true_set - pred_set)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    feedback = f"🔍 `{field}` partial match."
    feedback += f"- Correctly included: `{list(true_set & pred_set)}`\n"
    if fp:
        feedback += f"- Incorrectly included: `{list(pred_set - true_set)}`\n"
    if fn:
        feedback += f"- Missed: `{list(true_set - pred_set)}`"

    return f1, feedback.strip()

def feedback_e_issn(field, true_val, pred_val, p_issn_val):
    if true_val == pred_val:
        return 1.0, f"✅ `{field}` is correct: `{true_val}`."
    elif p_issn_val and pred_val == p_issn_val and true_val is None:
        return 1.0, f"✅ `{field}` is correctly inferred from `p_issn`: `{pred_val}`."
    else:
        return 0.0, f"❌ `{field}` is incorrect. You predicted `{pred_val}`, but the correct value is `{true_val}`."

def metadata_metric_with_feedback(example, pred, trace=None, pred_name=None, pred_trace=None):
    fields = [
        'language', 'title', 'creator', 'year', 'publisher',
        'doi', 'e_isbn', 'p_isbn', 'e_issn', 'p_issn', 'type_coar'
    ]

    scores = []
    feedback_parts = []

    metadata = json.loads(example.get("metadata", "{}"))
    ground_truth = example.get("ground_truth", {})

    for field in fields:
        true_val = metadata.get(field)
        pred_val = pred.get(field) or None

        if field in ['language', 'year', 'doi', 'p_issn', 'type_coar']:
            score, feedback = feedback_simple_string(field, true_val, pred_val)
        elif field == 'title':
            score, feedback = feedback_fuzzy_string(field, true_val, pred_val)
        elif field in ['creator', 'publisher', 'e_isbn', 'p_isbn']:
            score, feedback = feedback_set(field, true_val, pred_val)
        elif field == 'e_issn':
            p_issn_val = ground_truth.get("p_issn")
            score, feedback = feedback_e_issn(field, true_val, pred_val, p_issn_val)
        else:
            score, feedback = feedback_simple_string(field, true_val, pred_val)

        scores.append(score)
        feedback_parts.append(feedback)

    overall_score = sum(scores) / len(scores) if scores else 0
    full_feedback = "\n".join(feedback_parts)

    return dspy.Prediction(score=overall_score, feedback=full_feedback)


In [7]:
from dspy import GEPA

optimizer = GEPA(
    metric=metadata_metric_with_feedback,
    auto="heavy",
    num_threads=64,
    track_stats=False,
    use_merge=True,
    reflection_lm=reflection_lm
)

In [8]:
%%time

optimized_program = optimizer.compile(
    module,
    trainset=train_set,
    valset=val_set,
)

2025/09/30 09:18:15 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 1483 metric calls of the program. This amounts to 2.32 full evals on the train+val set.
2025/09/30 09:18:15 INFO dspy.teleprompt.gepa.gepa: Using 64 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget.
GEPA Optimization:   0%|          | 0/1483 [00:00<?, ?rollouts/s]2025/09/30 09:18:15 INFO dspy.evaluate.evaluate: Average Metric: 38.1608225108225 / 64 (59.6%)
2025/09/30 09:18:15 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.5962628517316018
GEPA Optimization:   4%|▍         | 64/1483 [00:00<00:15, 91.89rollouts/s]2025/09/30 09:18:15 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.5962628517316018


Average Metric: 1.91 / 3 (63.6%): 100%|██████████| 3/3 [00:00<00:00, 111.80it/s]

2025/09/30 09:18:15 INFO dspy.evaluate.evaluate: Average Metric: 1.9090909090909092 / 3 (63.6%)


2025/09/30 09:19:35 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for predict: **Task**: Extract structured bibliographic metadata from the JSON representation of a PDF document.  
The input is a JSON object with two top‑level keys:

* `pdfinfo` – metadata extracted from the PDF file (may contain `title`, `author`, `creationDate`, `modDate`, etc.).
* `pages` – a list of page objects, each with `page` (number) and `text` (the OCR‑extracted plain text of that page).

Your job is to produce **one JSON object** containing the fields listed below.  
If a field cannot be determined, use the exact empty value specified (e.g. `null` for a missing string, `[]` for an empty list).

---

### Output JSON Schema

| Field | Type | Description | Required format |
|-------|------|-------------|-----------------|
| `language` | string | ISO‑639‑1 language code of the document. Detect from the text: if the text contains Finnish‑specific characters (ä, ö, Ä, Ö, å, Å) or obvious Finnish w

Average Metric: 1.73 / 3 (57.6%): 100%|██████████| 3/3 [00:07<00:00,  2.33s/it]

...

2025/09/30 10:23:50 INFO dspy.teleprompt.gepa.gepa: Iteration 29: Proposed new text for predict: markdown
# Bibliographic‑Metadata Extraction – Revised Assistant Instructions

You will receive **one JSON object** that represents a PDF file.  
Its top‑level keys are:

* `pdfinfo` – metadata extracted from the PDF (may contain `title`, `author`,
  `creationDate`, `modDate`, …).  
* `pages` – an ordered list of page objects, each with:
  * `page` – page number (integer)  
  * `text` – the OCR‑extracted plain‑text of that page (UTF‑8 string)

Your job is to produce **exactly one JSON object** that follows the schema below.
If a value cannot be determined, use the exact empty value specified
(`null` for a missing string, `[]` for an empty list).

---

## Output JSON Schema (field order matters)

| Field | Type | Required format / notes |
|-------|------|--------------------------|
| `language` | string | `"fi"` if the document is Finnish, otherwise `"en"`. Detect by scanning **only the firs

Average Metric: 1.00 / 3 (33.3%): 100%|██████████| 3/3 [00:07<00:00,  2.34s/it]

2025/09/30 10:24:57 INFO dspy.evaluate.evaluate: Average Metric: 1.0 / 3 (33.3%)


2025/09/30 10:26:31 INFO dspy.teleprompt.gepa.gepa: Iteration 30: Proposed new text for predict: markdown
# Revised Task: Bibliographic Metadata Extraction from PDF‑derived JSON

You will receive **one** JSON object that represents the raw text extracted from a PDF document.  
The object always contains the top‑level keys **`pdfinfo`** and **`pages`** as described below.

## Input Structure

| Key | Type | Description |
|-----|------|-------------|
| `pdfinfo` | object | Metadata that the PDF file itself provides. May include `title`, `author`, `creationDate`, `modDate`, `lang` (optional) and other entries. |
| `pages`   | list of objects | Each element represents a page and has: <br>`page` – page number (integer) <br>`text` – plain‑text of the page (line‑breaks preserved). |

## Goal

From the information in **both** `pdfinfo` **and** the page texts, produce a **single** JSON object that conforms exactly to the **Output Schema** below.  
Only the fields listed in the schema may appear

CPU times: user 28.6 s, sys: 7.59 s, total: 36.2 s
Wall time: 1h 9min 14s


In [9]:
for name, pred in optimized_program.named_predictors():
    print("================================")
    print(f"Predictor: {name}")
    print("================================")
    print("Prompt:")
    print(pred.signature.instructions)
    print("*********************************")

Predictor: predict
Prompt:
markdown
# 📋 Task – Structured Bibliographic Metadata Extraction (Re‑specified)

You will receive **one JSON object** that represents a PDF document.  
The object has exactly two top‑level keys:

| Key      | Description |
|----------|-------------|
| `pdfinfo`| Metadata that was extracted directly from the PDF file (e.g. `title`, `author`, `creationDate`, `modDate`). |
| `pages`  | A list of page objects. Each page object contains `page` (the page number) and `text` (the OCR‑extracted plain‑text of that page). |

Your job is to produce **one JSON object** that follows the schema below.  
If a field cannot be determined, use the exact empty value indicated (`null` for scalars, `[]` for lists).  
All string values must be plain ASCII – normalise quotes to `"` (or the apostrophe `’`), collapse multiple spaces to a single space, and trim leading/trailing whitespace.

---

## Output JSON Schema

| Field | Type | Required format / rules |
|-------|------|---------

In [10]:
%%time

evaluate = dspy.Evaluate(
    devset=test_set,
    metric=metadata_metric_with_feedback,
    num_threads=64,
    display_table=True,
    display_progress=True,
    provide_traceback=True
)

eval_result = evaluate(optimized_program)

Average Metric: 20.91 / 34 (61.5%):  19%|█▊        | 34/182 [00:41<01:48,  1.36it/s]

2025/09/30 10:29:41 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=1024. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.7)  if the reason for truncation is repetition.


Average Metric: 21.44 / 35 (61.3%):  19%|█▉        | 35/182 [00:43<02:38,  1.08s/it]

2025/09/30 10:29:42 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=1024. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.7)  if the reason for truncation is repetition.
2025/09/30 10:29:42 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=1024. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.7)  if the reason for truncation is repetition.


Average Metric: 22.53 / 37 (60.9%):  20%|█▉        | 36/182 [00:44<02:10,  1.12it/s]

...

Average Metric: 113.03 / 180 (62.8%):  99%|█████████▉| 180/182 [02:22<00:00,  2.70it/s]

2025/09/30 10:31:22 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=1024. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.7)  if the reason for truncation is repetition.
2025/09/30 10:31:22 ERROR dspy.utils.parallelizer: Error for Example({'content': '{"pdfinfo": {"title": "TEAviisari", "creationDate": "D:20201219234023+02\'00\'", "modDate": "D:20201220002847+02\'00\'"}, "pages": [{"page": 1, "text": "#### TEAviisari\\n# Terveytt\\u00e4 edist\\u00e4v\\u00e4 liikunta ETEL\\u00c4-SAVON KUNNISSA 2020\\nLiikunnan edist\\u00e4misaktiivisuus Suomen kunnissa\\nTEAviisari 2020 -tiedonkeruussa\\nHyv\\u00e4 tulos*\\nParannettavaa\\nTieto puuttuu\\n\\n\\n- Pistem\\u00e4\\u00e4r\\u00e4 yli 75, kun tavoite 100. N\\u00e4in edellytykset\\n\\n\\nliikunnan edist\\u00e4miseen kunnassa ovat kaikilta osin\\

Average Metric: 113.57 / 181 (62.7%): 100%|██████████| 182/182 [02:24<00:00,  1.26it/s]

2025/09/30 10:31:22 INFO dspy.evaluate.evaluate: Average Metric: 113.57045454545464 / 182 (62.4%)
2025/09/30 10:31:22 WARNING dspy.evaluate.evaluate: Skipping table display since `pandas` is not installed.



CPU times: user 3.87 s, sys: 682 ms, total: 4.55 s
Wall time: 2min 24s


In [11]:
lm.inspect_history()





[2025-09-30T10:31:22.596853]

System message:

Your input fields are:
1. `content` (str):
Your output fields are:
1. `reasoning` (str): 
2. `language` (str): The language of the resource expressed as a BCP47 language tag.
3. `title` (str): The main title of the publication.
4. `alt_title` (list[str]): Alternative or parallel titles of the publication, suffixed with a BCP47 language tag in curly brackets.
5. `creator` (list[str]): The primary author(s) of the resource (order: Last Name, First Names).
6. `year` (Union[str, NoneType]): The year on which the resource was issued or made available.
7. `publisher` (list[str]): The entity/entities responsible for making the resource available.
8. `doi` (Union[str, NoneType]): The Digital Object Identifier (DOI) associated with the resource.
9. `e_isbn` (list[str]): The ISBN associated with the electronic resource.
10. `p_isbn` (list[str]): The ISBN of the printed version of this document.
11. `e_issn` (Union[str, NoneType]): The ISSN assoc

In [12]:
# save the optimized program for later use (many formats, just in case)
optimized_program.save("gepa-optimized-module.json", save_program=False)
optimized_program.save("gepa-optimized-module.pkl", save_program=False)
# save just the prompt(s)
for name, pred in optimized_program.named_predictors():
    with open(f"gepa-optimized-prompt-{name}.txt", "w") as outfile:
        outfile.write(pred.signature.instructions)
